In [58]:
# Load libraries
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import os

# NLTK
import nltk

# keras
import keras
from keras.models import Sequential
from keras.layers import SimpleRNN, Dropout, Flatten, Dense, BatchNormalization, LSTM, Embedding, Bidirectional, GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint

# text generation with LSTM experiments

To sample from a varying stochastic distribution let's create a reweight distribution function that uses "temperature" to push the entropy either up or down.

In [59]:
path = keras.utils.get_file('nietzsche.txt', origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600901


Now we create overlapping sequences (sliding windows) of sequences of characters. We get new sequences every 3 characters.

In [60]:
maxlen = 60 #size of sequence
step = 3 #we get a new sequence every 3 characters

sentences = [] #this will hold the sequences that we extract
next_chars = [] #this will hold the targets - the 'correct' characters that we are going to train the model on

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('# of sequences:', len(sentences))

chars = sorted(list(set(text))) # all the unique characters in the text since this is the output of softmax layer
print("# of unique chars:", len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) #get key-value maps for characters and index

print("vectorizing sequences..")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# of sequences: 200281
# of unique chars: 59
vectorizing sequences..


In [65]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [57]:
x.shape

(200281, 60, 59)

Below is a single 1 layer LSTM model. However, I initiated it with the Keras functional API instead of the normal sequential adding models style.

In [45]:
from keras import layers, Input
from keras.models import Model

input_tensor = Input(shape = (x.shape[1], x.shape[2]))
lstm1 = layers.LSTM(128, activation = 'relu')(input_tensor)
output_tensor = layers.Dense(len(chars), activation = 'softmax')(lstm1)

model = Model(input_tensor, output_tensor)
model.summary()

optimizer = keras.optimizers.RMSprop(lr=0.01)

model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 60, 59)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 128)               96256     
_________________________________________________________________
dense_8 (Dense)              (None, 59)                7611      
Total params: 103,867
Trainable params: 103,867
Non-trainable params: 0
_________________________________________________________________


Now after we train the model and inputting some seeding text we can generate new text by following a few steps in a loop:

1. draw from a probability distribution for the next character given the seed text and the already generated text.
2. reweigh the distribution to a certain temperatue 
3. sample the next character stochastically with the new distribution
4. add the new character to the sequence and repeat

In [66]:
#first we define a function to reweigh the distribution
def reweight_distribtution(original_dist, temperature = 0.5):
    distribution = np.log(original_dist) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

#and a function to sample the next character given the prediction of the model
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)
    

Now we set upt he training loop

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size = 128, epochs = 1) #fits model for 1 iteration
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen] #finds a random part of the text as a seed for the generator
    print(" --- Generating with seed: ---begin---" + generated_text + "---end---")
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature for this epoch:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400): #this part generates 400 characters with as start the seed-text
            sampled = np.zeros((1, maxlen, len(chars))) 
            for t, char in enumerate(generated_text): #here we one-hot encode the already generated characters
                sampled[0, t, char_indices[char]] = 1
                
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text  += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
    

epoch 1
Epoch 1/1
200281/200281 [==============================] - 240s 1ms/step - loss: 13.6625
 --- Generating with seed: ---begin---uffering--know ye not that it is only this
discipline that h---end---
------ temperature for this epoch: 0.2
uffering--know ye not that it is only this
discipline that h                                            

C:\Users\Laurens\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


                                                                                                                                                                                                                                                                                                                                                                    ------ temperature for this epoch: 0.5
                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ------ temperature for this epoch: 1.0
                                                                                                          

In [53]:
generated_text

'                                                           '